In [15]:
import pandas as pd
import numpy as np
import networkx as nx
from pyhive import hive
import pydoop.hdfs as hdfs
from pyhive import hive
from impala.dbapi import connect
from impala.util import as_pandas  

In [69]:
# Establishing the connection with Hive
host_name = "10.107.6.14"
port = 10000
user = "hdfs"
password = "bigid#01"
database="cx_test_ro_data"
conn = hive.Connection(host=host_name, port=port, username=user, password=password,
                           database=database, auth='CUSTOM')
cursor_1 = conn.cursor()

In [17]:
cursor_1.execute('select * from referral_count')
referral_count = as_pandas(cursor_1)

In [18]:
referral_count.shape

(585448, 2)

In [21]:
referral_count.head(4)

,refl_vin,refrl_count
0,...........544489,2
1,...........566698,2
2,..........E233621,2
3,.........YG695444,2


In [20]:
referral_count.rename(columns = {'_c1':'refrl_count'}, inplace=True)

In [25]:
# Reading the customer file 
cust_del_sat = pd.read_csv("cust_entire_details.csv")

/usr/local/src/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6,7,9,13,15,25,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
# Removing unnecessary columns
cust_del_sat.drop("Unnamed: 0", axis = 1, inplace = True)

In [28]:
print(cust_del_sat.shape)
cust_del_sat.head(3)

(1137734, 47)


,VIN Number,Model Name,Sale Date,Service Dealer,Ro Date,Max Milg,VISIT CNT,Customer Name,Latest Mobile Number,Ro Number,...,Region,Zone,Model Name.1,JDP/NonJDP,Year,customer_category,JDP_or_NonJDP,repm_vin,total_bill,RO_visit_count
0,MALA151ALEM335998,EON,42002,N4209,42339,2516,NaN,AJEET SINGH,8302895886,R201509204,...,N2,NaN,EON,JDP,2016,Satisfied,JDP,MALA151ALEM335998,4735.98,4
1,MALA151ALFM369430,EON,42091,N3405,42339,10105,NaN,MOHAMMAD AHETOSHAMMUL HAQ QURE,7417474576,R201514221,...,C3,NaN,EON,JDP,2016,Satisfied,JDP,MALA151ALFM369430,1613.51,4
2,MALA251ALEM271762,EON,41881,S5200,42339,5657,NaN,G RAM REDDY,7416851119,R201520587,...,S2,NaN,EON,JDP,2016,Delight,JDP,MALA251ALEM271762,31190.87,6


In [29]:
merged_df1 = pd.merge(cust_del_sat, referral_count[["refl_vin","refrl_count"]], left_on = "VIN Number", right_on = 'refl_vin')
merged_df1

,VIN Number,Model Name,Sale Date,Service Dealer,Ro Date,Max Milg,VISIT CNT,Customer Name,Latest Mobile Number,Ro Number,...,Model Name.1,JDP/NonJDP,Year,customer_category,JDP_or_NonJDP,repm_vin,total_bill,RO_visit_count,refl_vin,refrl_count
0,MALA351FLEM293041,EON,41848,S5208,42339,23167,NaN,PRAVEEN BABU,9246617786,R201517557,...,EON,JDP,2016,Delight,JDP,MALA351FLEM293041,110241.88,12,MALA351FLEM293041,2
1,MALA851DLEM106564,GRAND i10 (D),41819,N2230,42339,39957,NaN,FGIICL C O SATDEV,9999021688,R201502697,...,GRAND i10 (D),JDP,2016,Delight,JDP,MALA851DLEM106564,134885.01,32,MALA851DLEM106564,4
2,MALBM51RLFM060341,Elite i20 (1.4D),42045,N3425,42339,20357,NaN,HARISH KUMAR SHAH,9793035418,R201504871,...,Elite i20 (1.4D),JDP,2016,Delight,JDP,MALBM51RLFM060341,173385.82,20,MALBM51RLFM060341,2
3,MALBM51RLFM086188,Elite i20 (1.4D),42088,N6225,42339,8745,NaN,TAJINDERPAL,8054008700,R201508075,...,Elite i20 (1.4D),JDP,2016,Delight,JDP,MALBM51RLFM086188,145864.58,43,MALBM51RLFM086188,2
4,MALC381UMFM017536,CRETA (D),42216,E7405,42339,10130,NaN,BAMADEB ROUTRAY,7205471837,R201505565,...,CRETA (D),JDP,2016,Delight,JDP,MALC381UMFM017536,3128170.29,301,MALC381UMFM017536,2
5,MALC381UMFM017536,Creta,42216.0,E7405,NaN,NaN,NaN,BAMADEB ROUTRAY,9439431416,NaN,...,Creta,JDP,2017,Delight,JDP,MALC381UMFM017536,3128170.29,301,MALC381UMFM017536,2
6,MALC381UMFM017536,Creta,42216.0,E7405,NaN,NaN,NaN,BAMADEB ROUTRAY,9439431416,NaN,...,Creta,JDP,2017,Delight,JDP,MALC381UMFM017536,3128170.29,301,MALC381UMFM017536,2
7,MALC381UMFM017536,Creta,42216,E7405,NaN,NaN,NaN,BAMADEB ROUTRAY,9439431416,R201800572,...,Creta,JDP,2018,Delight,JDP,MALC381UMFM017536,3128170.29,301,MALC381UMFM017536,2
8,MALC381UMFM017536,Creta,NaN,E7405,43527.740625,NaN,NaN,BAMADEB ROUTRAY,NaN,R201901475,...,Creta,Non-JDP,2019,Satisfied,Non JDP,MALC381UMFM017536,3128170.29,301,MALC381UMFM017536,2
9,MALC381UMFM017536,Creta,NaN,E7405,43618.80287037037,NaN,NaN,BAMADEB ROUTRAY,NaN,R201903625,...,Creta,JDP,2019,Satisfied,JDP,MALC381UMFM017536,3128170.29,301,MALC381UMFM017536,2


In [34]:
merged_df1.shape

(196603, 49)

In [31]:
merged_df2 = pd.merge(cust_del_sat, referral_count[["refl_vin","refrl_count"]], how = 'left', left_on = "VIN Number", right_on = 'refl_vin')
merged_df2.shape

(1137734, 49)

In [32]:
merged_df2.head(4)

,VIN Number,Model Name,Sale Date,Service Dealer,Ro Date,Max Milg,VISIT CNT,Customer Name,Latest Mobile Number,Ro Number,...,Model Name.1,JDP/NonJDP,Year,customer_category,JDP_or_NonJDP,repm_vin,total_bill,RO_visit_count,refl_vin,refrl_count
0,MALA151ALEM335998,EON,42002,N4209,42339,2516,NaN,AJEET SINGH,8302895886,R201509204,...,EON,JDP,2016,Satisfied,JDP,MALA151ALEM335998,4735.98,4,NaN,NaN
1,MALA151ALFM369430,EON,42091,N3405,42339,10105,NaN,MOHAMMAD AHETOSHAMMUL HAQ QURE,7417474576,R201514221,...,EON,JDP,2016,Satisfied,JDP,MALA151ALFM369430,1613.51,4,NaN,NaN
2,MALA251ALEM271762,EON,41881,S5200,42339,5657,NaN,G RAM REDDY,7416851119,R201520587,...,EON,JDP,2016,Delight,JDP,MALA251ALEM271762,31190.87,6,NaN,NaN
3,MALA251ALEM298633,EON,41820,S7202,42339,32832,NaN,HASHIM T H,9037279323,R201506846,...,EON,JDP,2016,Delight,JDP,MALA251ALEM298633,20076.43,11,NaN,NaN


In [37]:
merged_df2.columns.tolist()

['VIN Number',
 'Model Name',
 'Sale Date',
 'Service Dealer',
 'Ro Date',
 'Max Milg',
 'VISIT CNT',
 'Customer Name',
 'Latest Mobile Number',
 'Ro Number',
 'Work Type',
 'Closer Date',
 'City Name',
 'State',
 'Pin Code',
 'Dealer Name',
 'Dealer Code',
 'availed_Pick_Drop_Facility',
 'Q1',
 'Q2',
 'Q3',
 'Q4',
 'Q5',
 'Q6',
 'For 1~8 Rating VOC',
 'Q15',
 'Overall Remarks',
 'Customer Comment',
 'Complaint',
 'Survey Disposition',
 'Campaign',
 'Service Advisor',
 '1-8 Rating',
 'Delight',
 'Dealer Name.1',
 'Top / Emerge',
 'City',
 'Region',
 'Zone',
 'Model Name.1',
 'JDP/NonJDP',
 'Year',
 'customer_category',
 'JDP_or_NonJDP',
 'repm_vin',
 'total_bill',
 'RO_visit_count',
 'refl_vin',
 'refrl_count']

In [38]:
merged_df2.drop(['repm_vin', 'refl_vin'], axis = 1, inplace = True)

In [40]:
merged_df2.shape

(1137734, 47)

In [41]:
merged_df2.to_csv('cust_refrl_count.csv', encoding = 'utf-8')

In [45]:
cursor_1.execute('select * from retail_count')
retail_count = as_pandas(cursor_1)

In [52]:
print(retail_count.shape)
retail_count.head(3)

(136270, 2)


,refl_vin,Total_retail_count
0,00000000000000132,1
1,00000000000000926,1
2,00000000000002016,1


In [49]:
retail_count.rename(columns = {'_c1':'Total_retail_count'}, inplace=True)

In [53]:
final_df = pd.merge(merged_df2, retail_count[['refl_vin', 'Total_retail_count']], how = 'left', left_on = "VIN Number", right_on = 'refl_vin')
final_df.shape

(1137734, 49)

In [54]:
final_df.head(3)

,VIN Number,Model Name,Sale Date,Service Dealer,Ro Date,Max Milg,VISIT CNT,Customer Name,Latest Mobile Number,Ro Number,...,Model Name.1,JDP/NonJDP,Year,customer_category,JDP_or_NonJDP,total_bill,RO_visit_count,refrl_count,refl_vin,Total_retail_count
0,MALA151ALEM335998,EON,42002,N4209,42339,2516,NaN,AJEET SINGH,8302895886,R201509204,...,EON,JDP,2016,Satisfied,JDP,4735.98,4,NaN,NaN,NaN
1,MALA151ALFM369430,EON,42091,N3405,42339,10105,NaN,MOHAMMAD AHETOSHAMMUL HAQ QURE,7417474576,R201514221,...,EON,JDP,2016,Satisfied,JDP,1613.51,4,NaN,NaN,NaN
2,MALA251ALEM271762,EON,41881,S5200,42339,5657,NaN,G RAM REDDY,7416851119,R201520587,...,EON,JDP,2016,Delight,JDP,31190.87,6,NaN,NaN,NaN


In [55]:
# Final data frame with all the columns
final_df.to_csv('final_df.csv', encoding = 'utf-8')

In [59]:
final_df[final_df['refrl_count'] == 528.0]

,VIN Number,Model Name,Sale Date,Service Dealer,Ro Date,Max Milg,VISIT CNT,Customer Name,Latest Mobile Number,Ro Number,...,Model Name.1,JDP/NonJDP,Year,customer_category,JDP_or_NonJDP,total_bill,RO_visit_count,refrl_count,refl_vin,Total_retail_count
670332,MALJ381AMGM010077,Tucson,NaN,N2219,NaN,NaN,NaN,SAMRITHI MOTORS PVT LTDD,9.215e+09,NaN,...,Tucson,Non JDP,2017,Delight,Non JDP,61019.28,12,528.0,MALJ381AMGM010077,36.0


In [62]:
#final_df['Total_retail_count'].value_counts()

final_df[final_df['Total_retail_count'] >= 100].shape

(33, 49)

In [66]:
final_df[final_df['refrl_count'] >= 100].shape

(238, 49)

In [68]:
final_df[final_df['refrl_count'] >= 300]

,VIN Number,Model Name,Sale Date,Service Dealer,Ro Date,Max Milg,VISIT CNT,Customer Name,Latest Mobile Number,Ro Number,...,Model Name.1,JDP/NonJDP,Year,customer_category,JDP_or_NonJDP,total_bill,RO_visit_count,refrl_count,refl_vin,Total_retail_count
37799,MALA841DLEM051746,Xcent (D),41973,N3217,42377,19910,NaN,Florence Ephraim,9.9199e+09,R201600129,...,Xcent (D),Non JDP,2016,Delight,Non JDP,361072.36,146,606.0,MALA841DLEM051746,122.0
37800,MALA841DLEM051746,Xcent,NaN,N3217,43631.754108796296,NaN,NaN,FLORENCE EPHRAIM,NaN,R201903837,...,Xcent,Non-JDP,2019,Delight,Non JDP,361072.36,146,606.0,MALA841DLEM051746,122.0
90020,MALBM51RLFM079433,i20 Active(1.4D),42153,E4200,42530,9000,NaN,FROSTEES EXPORT (INDIA) PVT.LTD.,9.86422e+09,R201604345,...,i20 Active(1.4D),JDP,2016,Delight,JDP,1140668.20,136,366.0,MALBM51RLFM079433,106.0
90021,MALBM51RLFM079433,NaN,NaN,E4200,NaN,NaN,NaN,FROSTEES EXPORT INDIA PVT.LTD.,9.86422e+09,NaN,...,NaN,JDP,2017,Satisfied,JDP,1140668.20,136,366.0,MALBM51RLFM079433,106.0
90022,MALBM51RLFM079433,i20 Active,42153,E4200,NaN,NaN,NaN,FROSTEES EXPORT INDIA PVT.LTD.,9.86422e+09,R201800347,...,i20 Active,JDP,2018,Delight,JDP,1140668.20,136,366.0,MALBM51RLFM079433,106.0
90023,MALBM51RLFM079433,i20 Active,42153.0,E4200,43524.66,NaN,NaN,FROSTEES EXPORT INDIA PVT.LTD.,NaN,R201901344,...,i20 Active,JDP,2019,Delight,JDP,1140668.20,136,366.0,MALBM51RLFM079433,106.0
103761,MALAM51BLEM592942,i10,41952,W5216,42524,11571,NaN,VINEETA CHANNY,9.82605e+09,R201601996,...,i10,Non JDP,2016,Satisfied,Non JDP,156384.54,117,1026.0,MALAM51BLEM592942,117.0
103762,MALAM51BLEM592942,i10,41952,W5216,42559,12145,NaN,VINEETA CHANNY,9.82605e+09,R201602461,...,i10,Non JDP,2016,Delight,Non JDP,156384.54,117,1026.0,MALAM51BLEM592942,117.0
103763,MALAM51BLEM592942,i10,2014-09-11 00:00:00,W5216,2020-02-07 00:00:00,NaN,NaN,VINEETA CHANNY,NaN,R202000724,...,i10,Emerging City,2020,Delight,JDP,156384.54,117,1026.0,MALAM51BLEM592942,117.0
177532,MALBM51RLFM186517,i20 Active(1.4D),42368,N3411,42623,24158,NaN,ABHISHEK MANDAL,7458023494,R201610063,...,i20 Active(1.4D),JDP,2016,Delight,JDP,251396.76,104,442.0,MALBM51RLFM186517,59.0


In [57]:
final_df['refrl_count'].value_counts()

2.0       106233
1.0        39166
4.0        24300
3.0         8667
6.0         7012
8.0         2673
5.0         2666
10.0        1129
7.0         1015
12.0         606
9.0          469
14.0         350
16.0         279
11.0         184
18.0         168
20.0         160
13.0         128
15.0          96
22.0          76
24.0          67
17.0          64
30.0          63
26.0          59
28.0          58
19.0          46
34.0          46
32.0          37
23.0          37
21.0          36
40.0          30
           ...  
828.0          1
166.0          1
83.0           1
528.0          1
864.0          1
339.0          1
79.0           1
93.0           1
77.0           1
76.0           1
75.0           1
94.0           1
1189.0         1
74.0           1
640.0          1
416.0          1
357.0          1
108.0          1
113.0          1
114.0          1
65.0           1
130.0          1
140.0          1
933.0          1
141.0          1
306.0          1
244.0          1
151.0         

### Calculating the difference between total actual bill and total estimate amount

In [70]:
cursor_1.execute('select * from ro_estmt_amt')
ro_estmt_amt = as_pandas(cursor_1)

In [71]:
print(ro_estmt_amt.shape)
ro_estmt_amt.head(3)

(13468659, 3)


,repm_vin,repm_estmt_amt,model_name
0,MALET41VLAM002525,0.00,MALET41VLAM002525
1,MALET41VLAM002525,4000.00,MALET41VLAM002525
2,MALET41VLAM002525,0.00,MALET41VLAM002525


In [73]:
# Calculating total estimate amt for all VINs
ro_estmt_amt['total_estmt_amt'] = ro_estmt_amt.repm_vin.map(ro_estmt_amt.groupby('repm_vin')['repm_estmt_amt'].sum())

In [74]:
ro_estmt_amt.head(3)

,repm_vin,repm_estmt_amt,model_name,total_estmt_amt
0,MALET41VLAM002525,0.00,MALET41VLAM002525,180050.00
1,MALET41VLAM002525,4000.00,MALET41VLAM002525,180050.00
2,MALET41VLAM002525,0.00,MALET41VLAM002525,180050.00


In [76]:
# Removing duplicates
ro_estmt_amt_dup_rm = ro_estmt_amt.drop_duplicates(subset = 'repm_vin', keep = 'first')

In [78]:
print(ro_estmt_amt_dup_rm.shape)
ro_estmt_amt_dup_rm.head(3)

(782372, 4)


,repm_vin,repm_estmt_amt,model_name,total_estmt_amt
0,MALET41VLAM002525,0.00,MALET41VLAM002525,180050.00
37,MALET41VLAM002490,12000.00,MALET41VLAM002490,87000.00
53,MALEN41BR5M005932,0.00,MALEN41BR5M005932,12360.00


In [77]:
# Reading the delight/satisfied customer data with all relevant columns
cust_del_sat_all = pd.read_csv("df_with_all_columns.csv")
cust_del_sat_all.shape

(1137734, 51)

In [79]:
cust_del_sat_all.head(3)

,Unnamed: 0,VIN Number,Model Name,Sale Date,Service Dealer,Ro Date,Max Milg,VISIT CNT,Customer Name,Latest Mobile Number,...,JDP/NonJDP,Year,customer_category,JDP_or_NonJDP,total_bill,RO_visit_count,refrl_count,refl_vin,Total_retail_count,complaint_count
0,0,MALA151ALEM335998,EON,42002,N4209,42339,2516,NaN,AJEET SINGH,8302895886,...,JDP,2016,Satisfied,JDP,4735.98,4,NaN,NaN,NaN,NaN
1,1,MALA151ALFM369430,EON,42091,N3405,42339,10105,NaN,MOHAMMAD AHETOSHAMMUL HAQ QURE,7417474576,...,JDP,2016,Satisfied,JDP,1613.51,4,NaN,NaN,NaN,NaN
2,2,MALA251ALEM271762,EON,41881,S5200,42339,5657,NaN,G RAM REDDY,7416851119,...,JDP,2016,Delight,JDP,31190.87,6,NaN,NaN,NaN,NaN


In [80]:
merged_final = pd.merge(cust_del_sat_all, ro_estmt_amt_dup_rm[["repm_vin","total_estmt_amt"]], left_on = "VIN Number", right_on = 'repm_vin')
merged_final.head(3)

,Unnamed: 0,VIN Number,Model Name,Sale Date,Service Dealer,Ro Date,Max Milg,VISIT CNT,Customer Name,Latest Mobile Number,...,customer_category,JDP_or_NonJDP,total_bill,RO_visit_count,refrl_count,refl_vin,Total_retail_count,complaint_count,repm_vin,total_estmt_amt
0,0,MALA151ALEM335998,EON,42002,N4209,42339,2516,NaN,AJEET SINGH,8302895886,...,Satisfied,JDP,4735.98,4,NaN,NaN,NaN,NaN,MALA151ALEM335998,7850.00
1,1,MALA151ALFM369430,EON,42091,N3405,42339,10105,NaN,MOHAMMAD AHETOSHAMMUL HAQ QURE,7417474576,...,Satisfied,JDP,1613.51,4,NaN,NaN,NaN,NaN,MALA151ALFM369430,2000.00
2,2,MALA251ALEM271762,EON,41881,S5200,42339,5657,NaN,G RAM REDDY,7416851119,...,Delight,JDP,31190.87,6,NaN,NaN,NaN,NaN,MALA251ALEM271762,9100.00


In [81]:
merged_final.shape

(1137734, 53)

In [82]:
# Removing unwanted columns from the data frame
# Removing unnecessary columns
merged_final = merged_final.drop(["Unnamed: 0", "refl_vin", "repm_vin"], axis = 1)

merged_final.head(3)

,VIN Number,Model Name,Sale Date,Service Dealer,Ro Date,Max Milg,VISIT CNT,Customer Name,Latest Mobile Number,Ro Number,...,JDP/NonJDP,Year,customer_category,JDP_or_NonJDP,total_bill,RO_visit_count,refrl_count,Total_retail_count,complaint_count,total_estmt_amt
0,MALA151ALEM335998,EON,42002,N4209,42339,2516,NaN,AJEET SINGH,8302895886,R201509204,...,JDP,2016,Satisfied,JDP,4735.98,4,NaN,NaN,NaN,7850.00
1,MALA151ALFM369430,EON,42091,N3405,42339,10105,NaN,MOHAMMAD AHETOSHAMMUL HAQ QURE,7417474576,R201514221,...,JDP,2016,Satisfied,JDP,1613.51,4,NaN,NaN,NaN,2000.00
2,MALA251ALEM271762,EON,41881,S5200,42339,5657,NaN,G RAM REDDY,7416851119,R201520587,...,JDP,2016,Delight,JDP,31190.87,6,NaN,NaN,NaN,9100.00


In [84]:
merged_final.dtypes

VIN Number                     object
Model Name                     object
Sale Date                      object
Service Dealer                 object
Ro Date                        object
Max Milg                       object
VISIT CNT                      object
Customer Name                  object
Latest Mobile Number           object
Ro Number                      object
Work Type                      object
Closer Date                    object
City Name                      object
State                          object
Pin Code                       object
Dealer Name                    object
Dealer Code                    object
availed_Pick_Drop_Facility     object
Q1                            float64
Q2                            float64
Q3                            float64
Q4                            float64
Q5                            float64
Q6                            float64
For 1~8 Rating VOC             object
Q15                           float64
Overall Rema

In [85]:
# Changing the data type of total estmt amt to float
merged_final["total_estmt_amt"] = merged_final.total_estmt_amt.astype(float)

In [86]:
# Calculating the difference between the actual total bill and total estimate amount
merged_final['actual_estmt_diff'] = merged_final['total_bill'] - merged_final['total_estmt_amt'] 

In [87]:
# Where the diff is in negative, it means that the actual bill was less than the estimate amount
# while where the estmt diff is positive it means that the actual bill was more than the estimate amount
merged_final.head(3)

,VIN Number,Model Name,Sale Date,Service Dealer,Ro Date,Max Milg,VISIT CNT,Customer Name,Latest Mobile Number,Ro Number,...,Year,customer_category,JDP_or_NonJDP,total_bill,RO_visit_count,refrl_count,Total_retail_count,complaint_count,total_estmt_amt,actual_estmt_diff
0,MALA151ALEM335998,EON,42002,N4209,42339,2516,NaN,AJEET SINGH,8302895886,R201509204,...,2016,Satisfied,JDP,4735.98,4,NaN,NaN,NaN,7850.0,-3114.02
1,MALA151ALFM369430,EON,42091,N3405,42339,10105,NaN,MOHAMMAD AHETOSHAMMUL HAQ QURE,7417474576,R201514221,...,2016,Satisfied,JDP,1613.51,4,NaN,NaN,NaN,2000.0,-386.49
2,MALA251ALEM271762,EON,41881,S5200,42339,5657,NaN,G RAM REDDY,7416851119,R201520587,...,2016,Delight,JDP,31190.87,6,NaN,NaN,NaN,9100.0,22090.87


In [89]:
merged_final.to_csv('final_df_with_diff_col.csv', encoding = 'utf-8')

In [91]:
merged_final.head(3)

,VIN Number,Model Name,Sale Date,Service Dealer,Ro Date,Max Milg,VISIT CNT,Customer Name,Latest Mobile Number,Ro Number,...,Year,customer_category,JDP_or_NonJDP,total_bill,RO_visit_count,refrl_count,Total_retail_count,complaint_count,total_estmt_amt,actual_estmt_diff
0,MALA151ALEM335998,EON,42002,N4209,42339,2516,NaN,AJEET SINGH,8302895886,R201509204,...,2016,Satisfied,JDP,4735.98,4,NaN,NaN,NaN,7850.0,-3114.02
1,MALA151ALFM369430,EON,42091,N3405,42339,10105,NaN,MOHAMMAD AHETOSHAMMUL HAQ QURE,7417474576,R201514221,...,2016,Satisfied,JDP,1613.51,4,NaN,NaN,NaN,2000.0,-386.49
2,MALA251ALEM271762,EON,41881,S5200,42339,5657,NaN,G RAM REDDY,7416851119,R201520587,...,2016,Delight,JDP,31190.87,6,NaN,NaN,NaN,9100.0,22090.87


In [93]:
merged_final[merged_final['complaint_count'] == 6]

,VIN Number,Model Name,Sale Date,Service Dealer,Ro Date,Max Milg,VISIT CNT,Customer Name,Latest Mobile Number,Ro Number,...,Year,customer_category,JDP_or_NonJDP,total_bill,RO_visit_count,refrl_count,Total_retail_count,complaint_count,total_estmt_amt,actual_estmt_diff
49122,MALCT41RLFM188696,NEW VERNA,42067,N3414,42448,19035,NaN,SHALENDRA KUMAR SHRIVASTAVA(UP,9.87381e+09,R201601414,...,2016,Satisfied,JDP,455896.25,39,NaN,NaN,6.0,125300.0,330596.25
58733,MALA841DLEM036094,Xcent (D),41869,E6408,42481,24620,NaN,Kripa Nath Singh,9708900333,R201602103,...,2016,Delight,JDP,1249428.38,66,NaN,NaN,6.0,61300.0,1188128.38
58734,MALA841DLEM036094,Xcent (D),41869,E6408,42534,39171,NaN,Kripa Nath Singh,9708900333,R201603393,...,2016,Delight,JDP,1249428.38,66,NaN,NaN,6.0,61300.0,1188128.38
80931,MALA841CLEM041896,Xcent (P),41915,W1412,42514,19965,NaN,CDR PANKAJ BHATNAGAR,9225977077,R201603021,...,2016,Satisfied,JDP,139277.68,34,NaN,NaN,6.0,32800.0,106477.68
80932,MALA841CLEM041896,Xcent,NaN,W2406,43714.0,NaN,NaN,BHATNAGAR PANKAJ,NaN,R201909649,...,2019,Satisfied,JDP,139277.68,34,NaN,NaN,6.0,32800.0,106477.68
294604,MALA851CLGM452824,NaN,NaN,N2223,NaN,NaN,NaN,NITIN AGGARWAL,9958079094,NaN,...,2017,Satisfied,Non JDP,59554.00,72,2.0,NaN,6.0,70000.0,-10446.00
294605,MALA851CLGM452824,NaN,NaN,N2223,NaN,NaN,NaN,NITIN AGGARWAL,9958079094,NaN,...,2017,Satisfied,Non JDP,59554.00,72,2.0,NaN,6.0,70000.0,-10446.00
294606,MALA851CLGM452824,GRAND i10 (P),NaN,N1200,NaN,NaN,NaN,NITIN AGGARWAL,9650139060,NaN,...,2017,Satisfied,JDP,59554.00,72,2.0,NaN,6.0,70000.0,-10446.00
294607,MALA851CLGM452824,Grand i10,NaN,N1200,NaN,NaN,NaN,NITIN AGGARWAL,NaN,R201814570,...,2019,Satisfied,JDP,59554.00,72,2.0,NaN,6.0,70000.0,-10446.00
634363,MALA851CMHM679207,GRAND i10 (P),NaN,W2204,NaN,NaN,NaN,SHIVDE BHAGYASHREE SUBODH,9763704833,NaN,...,2017,Satisfied,JDP,24927.00,20,NaN,NaN,6.0,18000.0,6927.00


In [105]:
merged_final.tail(4)

,VIN Number,Model Name,Sale Date,Service Dealer,Ro Date,Max Milg,VISIT CNT,Customer Name,Latest Mobile Number,Ro Number,...,Year,customer_category,JDP_or_NonJDP,total_bill,RO_visit_count,refrl_count,Total_retail_count,complaint_count,total_estmt_amt,actual_estmt_diff
1137730,MALA741CLJM330592,Xcent (P),2018-10-16 00:00:00,W1221,2020-06-13 00:00:00,NaN,NaN,YASHWANT S. BATWALKAR,NaN,R202003500,...,2020,Delight,JDP,9261.02,8,NaN,NaN,1.0,0.0,9261.02
1137731,MALJ381ASJM013042,Tucson (TLi) D,2019-02-28 00:00:00,W5205,2020-06-12 00:00:00,NaN,NaN,ROHIT SATISH KALE .,NaN,R202003799,...,2020,Delight,JDP,889259.20,12,NaN,NaN,NaN,0.0,889259.20
1137732,MALD341CLHM012170,Elantra (ADi) P,2017-05-08 00:00:00,S1100,2020-06-15 00:00:00,NaN,NaN,RAJA,NaN,R202006500,...,2020,Delight,JDP,284476.46,21,NaN,NaN,NaN,26200.0,258276.46
1137733,MALSW81XSFM011983,SANTA FE (DMi),2015-03-26 00:00:00,W3222,2020-06-15 00:00:00,NaN,NaN,NAUSHAD MALIK,NaN,R202004221,...,2020,Delight,JDP,541451.65,32,NaN,NaN,NaN,23000.0,518451.65


In [108]:
merged_final.groupby('VIN Number')['customer_category'].sum()

VIN Number
KMHSH81XLAU667673                           Delight
KMHSH81XLBU724388                         Satisfied
KMHSH81XLBU738910                         Satisfied
KMHSH81XLBU741516                           Delight
KMHSH81XLBU753464                         Satisfied
KMHSH81XNAU666861                           Delight
KMHSH81XNAU667009                           Delight
KMHSH81XNAU700763                         Satisfied
KMHSH81XNAU701430                         Satisfied
KMHSH81XNBU702991                           Delight
KMHSH81XNBU703395                         Satisfied
KMHSH81XNBU703518                           Delight
KMHSH81XNBU724542                           Delight
KMHSH81XNBU738132                         Satisfied
KMHSH81XNBU738377                           Delight
KMHSH81XNBU741579                         Satisfied
KMHSH81XNBU741627                         Satisfied
KMHSH81XNBU744166                         Satisfied
KMHSH81XNBU744175                           Delight
K

In [109]:
merged_final[merged_final['VIN Number'] == 'MALSW81XSHM013539']

,VIN Number,Model Name,Sale Date,Service Dealer,Ro Date,Max Milg,VISIT CNT,Customer Name,Latest Mobile Number,Ro Number,...,Year,customer_category,JDP_or_NonJDP,total_bill,RO_visit_count,refrl_count,Total_retail_count,complaint_count,total_estmt_amt,actual_estmt_diff
651718,MALSW81XSHM013539,New Santa Fe (DM),NaN,N2229,NaN,NaN,NaN,TRIVALENT ENGINNERS AND CONSULTANTS PVT. LTD.,9818031173,NaN,...,2017,Delight,JDP,1565723.56,120,5.0,NaN,NaN,42000.0,1523723.56
651719,MALSW81XSHM013539,New Santa Fe (DM),42887,N2229,NaN,NaN,NaN,TRIVALENT ENGINNERS AND CONSULTANTS PVT. LTD.,9818031173,DL2CAX6363,...,2018,Delight,JDP,1565723.56,120,5.0,NaN,NaN,42000.0,1523723.56
651720,MALSW81XSHM013539,SANTA FE (DMi),42887,N2229,NaN,NaN,NaN,TRIVALENT ENGINNERS AND CONSULTANTS PVT. LTD.,9818031173,R201804916,...,2018,Satisfied,JDP,1565723.56,120,5.0,NaN,NaN,42000.0,1523723.56
651721,MALSW81XSHM013539,SANTA FE (DMi),2017-06-01 00:00:00,N2229,2020-02-18 00:00:00,NaN,NaN,TRIVALENT ENGINNERS AND CONSULTANTS PVT. LTD.,NaN,R202002058,...,2020,Delight,JDP,1565723.56,120,5.0,NaN,NaN,42000.0,1523723.56


### Grouping VINs based on customer category to check which VIN has multiple values for customer category

In [110]:
merged_final.groupby(['VIN Number','customer_category'])['customer_category'].count()

VIN Number         customer_category
KMHSH81XLAU667673  Delight              1
KMHSH81XLBU724388  Satisfied            1
KMHSH81XLBU738910  Satisfied            1
KMHSH81XLBU741516  Delight              1
KMHSH81XLBU753464  Satisfied            1
KMHSH81XNAU666861  Delight              1
KMHSH81XNAU667009  Delight              1
KMHSH81XNAU700763  Satisfied            1
KMHSH81XNAU701430  Satisfied            1
KMHSH81XNBU702991  Delight              1
KMHSH81XNBU703395  Satisfied            1
KMHSH81XNBU703518  Delight              1
KMHSH81XNBU724542  Delight              1
KMHSH81XNBU738132  Satisfied            1
KMHSH81XNBU738377  Delight              1
KMHSH81XNBU741579  Satisfied            1
KMHSH81XNBU741627  Satisfied            1
KMHSH81XNBU744166  Satisfied            1
KMHSH81XNBU744175  Delight              1
KMHSH81XNBU753195  Satisfied            1
KMHSH81XNBU766138  Delight              1
KMHSU81CSGU733630  Satisfied            1
MALA151ALBM012697  Satisfied           

In [111]:
fluctuating_customers = merged_final.groupby(['VIN Number','customer_category'])['customer_category'].count()

In [114]:
fluctuating_customers.head(4)

VIN Number         customer_category
KMHSH81XLAU667673  Delight              1
KMHSH81XLBU724388  Satisfied            1
KMHSH81XLBU738910  Satisfied            1
KMHSH81XLBU741516  Delight              1
Name: customer_category, dtype: int64